<a href="https://colab.research.google.com/github/Ahm1027/Payloads-Model/blob/Debug-Tests/Payload_Model_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk import ngrams
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from google.colab import drive
import binascii

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
def get_hexdump(executable, num_of_ngrams=4):
  executable = open(executable, 'rb')
  data = executable.read()
  executable.close()
  data2hex = bin2hex(data)
  output = ngrams(data2hex.split(" "), num_of_ngrams)
  return [ ''.join(grams) for grams in output]

def bin2hex(data):
  data2hex = ""
  for byte in data:
    hex = str(binascii.hexlify(bytes([byte]))).split("'")
    if (len(hex) >= 2):
      data2hex += hex[1]+" "
    else:
      data2hex += hex[0]+" "
  return data2hex

# def generate_ngrams(text, WordsToCombine):
#     words = text.split()
#     output = []  
#     for i in range(len(words)- WordsToCombine+1):
#       word = ''
#       for i in range (i, i+WordsToCombine):
#         word = word + words[i]
#       output.append(word)
#     return output

# def calculateTotalCounts(uniqueOutput, output):
#   Uniquecounts = []
#   for i in range(len(uniqueOutput)):
#     count = 0
#     for j in range(len(output)):
#       if output[j] == uniqueOutput[i]:
#         count = count+1
#     Uniquecounts.append(count)
#   return Uniquecounts

In [9]:
def vectorize(data):
  vectorizer = CountVectorizer()
  vectorizer.fit(data)
  vectorized = vectorizer.transform(data)
  return vectorizer, vectorized

In [10]:
def arrangeAndSort(data_vectorizer, data_vectorized):
  data_arranged = list(zip(data_vectorizer.get_feature_names(), data_vectorized.sum(0).getA1()))
  data_sorted = sorted(data_arranged, key=lambda item: item[1], reverse=True)
  return data_sorted

In [ ]:
def resolveMalwareFiles(FOLDER_PATHS, num_of_features=2000):
  stopCheck, prev=0, 0
  globalFeatures = np.array([])
  for files in os.listdir(FOLDER_PATHS):
    filepath = r"/content/drive/MyDrive/Payloads/"+files
    data = get_hexdump(filepath)
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:num_of_features]
    if (len(globalFeatures) == 0):
      globalFeatures = np.array(data_sorted)[:, 0].astype(str).tolist()
    else:
      globalFeatures = np.append(globalFeatures, np.array(data_sorted)[:, 0].astype(str).tolist())
      indexes = np.unique(globalFeatures, return_index=True)[1]
      globalFeatures = [globalFeatures[index] for index in sorted(indexes)]
    if (stopCheck == prev+10):
      prev = stopCheck
      print("%f percent completed"%((stopCheck/1000)*100), end=", ")
      print("Number of Features: %d"%(len(globalFeatures)))
      print("Number of files processed: %d"%(stopCheck))
    stopCheck += 1
  return globalFeatures

globalFeatures = resolveMalwareFiles(r"/content/drive/MyDrive/Payloads")
print(len(globalFeatures))

1.000000 percent completed, Number of Features: 12824
Number of files processed: 10
2.000000 percent completed, Number of Features: 21665
Number of files processed: 20
3.000000 percent completed, Number of Features: 32188
Number of files processed: 30
4.000000 percent completed, Number of Features: 39380
Number of files processed: 40
5.000000 percent completed, Number of Features: 42516
Number of files processed: 50
6.000000 percent completed, Number of Features: 48005
Number of files processed: 60
7.000000 percent completed, Number of Features: 53519
Number of files processed: 70
8.000000 percent completed, Number of Features: 57798
Number of files processed: 80
9.000000 percent completed, Number of Features: 61965
Number of files processed: 90
10.000000 percent completed, Number of Features: 66964
Number of files processed: 100
11.000000 percent completed, Number of Features: 69128
Number of files processed: 110
12.000000 percent completed, Number of Features: 71534
Number of files p

In [ ]:
len(globalFeatures)

412090

In [11]:
np.save('features_np.npy', globalFeatures)

NameError: ignored

In [12]:
globalFeatures = np.load(r'/content/features_np.npy')
len(globalFeatures)

412090

In [ ]:
def flatten_array(A):
    rt = []
    for i in A:
      if isinstance(i, np.ndarray):
        print('inside')
        rt.extend(flatten(i))
      else: rt.append(i)
    return np.array(rt)

globalFeatures = np.array(globalFeatures)
globalFeatures = np.sort(globalFeatures)
globalFeatures = flatten_array(globalFeatures)

In [ ]:
def normalizeFeatures(data):
  data_scaled = (data - data.min()) / (data.max() - data.min())   #MinMax Normalization
  return data_scaled

In [ ]:
# def find_first_no_jit(item, vec):
#   index = np.where(vec == item)
#   if index:
#     return index
#   return -1

def binarySearch(array, item):
  lowPoint = 0
  highpoint = len(array) - 1
  while (lowPoint <= highpoint):
      midPoint = int((lowPoint + highpoint) / 2)
      if item == array[midPoint]:
          return midPoint
      elif item < array[midPoint]:
          highpoint = midPoint - 1
      else:
          lowPoint = midPoint + 1

  return -1

def mergeWithGlobalVocab(globalFeatures):
  stopCheck, prev=0, 0
  global_Merged = []
  for files in os.listdir(r"/content/drive/MyDrive/Payloads"):
    filepath = r"/content/drive/MyDrive/Payloads/"+files
    data = get_hexdump(filepath)
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:2000]
    features = [0]*globalFeatures.size
    features = np.array(features)
    for x in data_sorted:
      id = binarySearch(globalFeatures, x[0])
      if (id != -1):
        features[id] = x[1]
    global_Merged.append(features)
    if (stopCheck == prev+10):
      prev = stopCheck
      print("%f percent completed"%((stopCheck/1080)*100), end=", ")
      print("Length of Merged Features: %d"%(len(global_Merged)), end=", ")
      print("Number of files processed: %d"%(stopCheck))
    stopCheck += 1
  return np.array(global_Merged)

merged_features = mergeWithGlobalVocab(globalFeatures)

0.925926 percent completed, Length of Merged Features: 11, Number of files processed: 10
1.851852 percent completed, Length of Merged Features: 21, Number of files processed: 20
2.777778 percent completed, Length of Merged Features: 31, Number of files processed: 30
3.703704 percent completed, Length of Merged Features: 41, Number of files processed: 40
4.629630 percent completed, Length of Merged Features: 51, Number of files processed: 50
5.555556 percent completed, Length of Merged Features: 61, Number of files processed: 60
6.481481 percent completed, Length of Merged Features: 71, Number of files processed: 70
7.407407 percent completed, Length of Merged Features: 81, Number of files processed: 80
8.333333 percent completed, Length of Merged Features: 91, Number of files processed: 90
9.259259 percent completed, Length of Merged Features: 101, Number of files processed: 100
10.185185 percent completed, Length of Merged Features: 111, Number of files processed: 110
11.111111 percen

In [ ]:
# compressed_SVD = TruncatedSVD(30)
# compressed_data = compressed_SVD.fit_transform(features_frequency)
# pd.DataFrame(compressed_SVD.components_)

In [ ]:
len(globalFeatures)

412090